In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Promos'

In [2]:
from pathlib import Path

from pandas import DataFrame, read_csv, to_datetime


class PromoResults:
    def __init__(self, base_dir: str) -> None:
        self.base_dir = Path(base_dir)
        self.stores_file = self.base_dir.joinpath('stores.csv')
        self.promo_file = self.base_dir.joinpath('promo_results.csv')
        for needed_file in [self.stores_file, self.promo_file]:
            if not needed_file.is_file():
                file_name = str(needed_file).split('/')[-1]
                print(f'There should be a file called "{file_name}" at:\n{self.base_dir}\nAdd it and try again!')
        self.orders_files = self.base_dir.glob('orders*.csv')

    
    def read_files(self) -> None:
        self.stores = read_csv(self.stores_file)
        self.promo = read_csv(self.promo_file, sep='\t', encoding='utf-16')
        self.orders = DataFrame()
        for file_chunk in self.orders_files:
            self.orders = self.orders.append(read_csv(file_chunk), ignore_index=True)


    def mod_stores_data(self) -> None:
        self.stores['store'] = self.stores['store'].map(lambda x: x.title())


    def mod_promo_data(self, date_cols: list) -> None:
        for col in date_cols:
            self.promo[col] = to_datetime(self.promo[col])


    def get_store(self, col_to_search: str, n_words: int) -> None:
        promo_copy = self.promo.copy()
        promo_copy['store'] = promo_copy[col_to_search].map(lambda x: ''.join(str(x).split()[:n_words]).title())
        promo_copy = promo_copy.merge(self.stores, on='store', how='left')
        promo_null = promo_copy[promo_copy[self.stores.columns[-1]].isnull()][self.promo.columns].copy()
        promo_copy.dropna(subset=[self.stores.columns[-1]], inplace=True)

        promo_null['store'] = promo_null[col_to_search].map(lambda x: ' '.join(str(x).split()[:n_words+1]).title())
        promo_null = promo_null.merge(self.stores, on='store', how='left')
        promo_copy = promo_copy.append(promo_null)

        self.promo = promo_copy.copy()


    def query_orders(self, start_date_col: str, end_date_col: str) -> DataFrame:
        

pr = PromoResults(BASE_DIR)
pr.read_files()
pr.mod_stores_data()
pr.mod_promo_data(date_cols=['project valid from', 'project valid until'])
pr.get_store(col_to_search='project name', n_words=1)
pr.promo.head()

,CPG,project ID,project name,project valid from,project valid until,project orders,budget used,project duration (days),completed days (%),sales,...,units delivered,project unique users,project running (days),store,store_id,store_category_id,store_category,store_family,store_type,store_group
0,Partners (Dummy),34647,Soriana Vinos 13-18 may 22,2022-05-13 17:20:06,2022-05-19 01:00:00,34,"1,252.68",6,17%,"2,147.00",...,13,34,1,Soriana,"3,729",13.0,Supermercados,Supermercados,Groceries,Supermarket
1,Partners (Dummy),34645,Soriana Estelares 13-16 May 22,2022-05-13 17:16:00,2022-05-17 01:00:00,345,"5,494.66",4,25%,"8,387.27",...,231,345,1,Soriana,"3,729",13.0,Supermercados,Supermercados,Groceries,Supermarket
2,Partners (Dummy),34608,Chedraui - FDS 13 - 16 Mayo,2022-05-13 05:00:00,2022-05-16 01:00:00,617,"16,052.89",3,33%,"31,624.65",...,792,612,1,Chedraui,22,13.0,Supermercados,Supermercados,Groceries,Supermarket
3,Partners (Dummy),34604,Soriana Electrodomésticos 13-17 de Mayo,2022-05-13 15:00:00,2022-05-18 00:00:00,2,679.60,5,20%,"1,699.00",...,1,2,1,Soriana,"3,729",13.0,Supermercados,Supermercados,Groceries,Supermarket
4,Partners (Dummy),34581,HEB Carnes 13-19 Mayo 22,2022-05-13 08:00:00,2022-05-20 01:00:00,199,"5,195.97",7,14%,"9,699.43",...,107,199,1,Heb,25,13.0,Supermercados,Supermercados,Groceries,Supermarket


In [10]:
pr.promo[pr.promo['store_id'].isnull()]#['store'].value_counts()

,CPG,project ID,project name,project valid from,project valid until,project orders,budget used,project duration (days),completed days (%),sales,...,units delivered,project unique users,project running (days),store,store_id,store_category_id,store_category,store_family,store_type,store_group
16,Partners (Dummy),34525,Campaña HEB cupón 12-16 Mayo,2022-12-05 05:00:00,2022-05-17 05:00:00,76,"22,800.00",5,40%,"181,835.01",...,"3,207",75,2,Campaña Heb,NaN,NaN,NaN,NaN,NaN,NaN
17,Partners (Dummy),34480,KK Promo: 25% Docena OG Días: 12 a 15 May,2022-12-05 05:00:00,2022-05-16 05:00:00,123,"10,520.25",4,50%,"31,623.00",...,127,122,2,Kk Promo:,NaN,NaN,NaN,NaN,NaN,NaN
18,Partners (Dummy),34430,FM Promos mensuales Mayo 22,2022-11-05 05:00:00,2022-01-06 05:00:00,7,502.31,21,14%,"2,195.60",...,44,7,3,Fm Promos,NaN,NaN,NaN,NaN,NaN,NaN
21,Partners (Dummy),34312,Collection Pharmas W19 Alsuper Farmacia 09-15 ...,2022-09-05 05:00:00,2022-05-16 05:00:00,4,540.00,7,71%,"2,925.50",...,15,4,5,Collection Pharmas,NaN,NaN,NaN,NaN,NaN,NaN
22,Partners (Dummy),34311,Collection Pharmas W19 Costco Farmacia 09-15 mayo,2022-09-05 05:00:00,2022-05-16 05:00:00,18,"3,320.00",7,71%,"12,771.91",...,25,18,5,Collection Pharmas,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,Partners (Dummy),28620,Laika promo miercoles 10%off 05ene22,2022-05-01 06:00:00,2022-06-01 06:00:00,4,321.50,1,100%,"3,215.00",...,11,4,1,Laika Promo,NaN,NaN,NaN,NaN,NaN,NaN
319,Partners (Dummy),28441,The Body Shop promo 26-08ene22,2021-12-29 15:00:00,2022-09-01 03:00:00,31,"6,822.00",11,91%,"18,900.00",...,44,31,10,The Body,NaN,NaN,NaN,NaN,NaN,NaN
336,Partners (Dummy),28312,THD Cajas 23dic- 12ene22,2021-12-23 14:00:00,2022-01-13 03:00:00,31,"7,558.00",21,95%,"32,199.00",...,133,28,20,Thd Cajas,NaN,NaN,NaN,NaN,NaN,NaN
338,Partners (Dummy),28278,THD promos nxm 23 Dic - 12 Ene21,2021-12-23 06:00:00,2022-01-13 06:00:00,1,535.00,21,71%,"2,140.00",...,4,1,15,Thd Promos,NaN,NaN,NaN,NaN,NaN,NaN
